# A brief introduction to using Python in Astronomy

Efficient use of Python in any data-intensive application requires the appropriate use of additional Python libraries.
This notebook provides a quick introduction to (mostly) `numpy` and `scipy`, which are commonly used in scientific Python, and to `astropy`, which is very helpful for astronomy.

## NumPy

### Motivation

A Python `list` can hold elements of different data types.
Operations can be defined in a meaningful way for many different datatypes, though with different outcomes.

In [ ]:
for elem in (5, "5", [5]):
    print(f"If {elem} is a {type(elem)} instance, then {2*elem = }")

In scientific computing it is very often necessary to perform a large number of analogous operations with many numerical values.
Using a `list` for storing numbers in resource-intensive calculations means that the Python interpreter needs to check every single one of them to determine if it is indeed numerical and what an arithmetic operation performed with it means.
This overhead slows the code down.
The way to get around this limitation is to write vectorized code with the [NumPy](https://numpy.org/) package.

We shall first illustrate the speed difference between basic Python and `numpy` by implementing functions that compute pairwise differences between numbers.
A more thorough explanation on how to vectorize code using `numpy` will be presented below.

We will use the [IPython `%timeit` magic function](https://ipython.readthedocs.io/en/stable/interactive/magics.html) to time the code execution.
This will work in a Jupyter notebook but not in the basic CPython interpreter.

In [ ]:
def pairwise_differences_with_for_loops(arr):
    diffs = []
    for elem1 in arr:
        diffs.append([])
        for elem2 in arr:
            diffs[-1].append(elem1 - elem2)
    return diffs


def pairwise_differences_with_list_comprehensions(arr):
    return [[elem1 - elem2 for elem2 in arr] for elem1 in arr]


small_list = [1, 4, 9]
print(pairwise_differences_with_for_loops(small_list))
large_list = list(range(1000))
%timeit pairwise_differences_with_for_loops(large_list)

print(pairwise_differences_with_list_comprehensions(small_list))
%timeit pairwise_differences_with_list_comprehensions(large_list)

In [ ]:
import numpy as np


def pairwise_differences_with_numpy(arr):
    return arr[:, np.newaxis] - arr


small_array = np.array(small_list)
diffs = pairwise_differences_with_numpy(small_array)
print(diffs)
large_array = np.array(large_list)
%timeit pairwise_differences_with_numpy(large_array)

We can see that using list comprehensions is roughly twice as fast as using for-loops, but using `numpy` is roughly 50 times faster still.

### Arrays

One of the key concepts that allows `numpy` to perform so much better is the `numpy` array.
An array is a collection of elements of the same datatype that has some size, i.e. total number of elements in it, and some number of dimensions or axes, which is the number of indices required to identify an element.
An array also has a shape, which is the size of the array along all the different axes.

In [ ]:
for arr in (small_array, diffs):
    print(
        f"Array:\n{arr}",
        f"{arr.size = }",
        f"{arr.ndim = }",
        f"{arr.shape = }",
        sep="\n",
    )
    print()

An array can be generated from a `list` as we have done above, it can be initialized with some default value or it can be obtained by manipulating other arrays.

In [ ]:
false_arr = np.zeros(5, dtype=bool)
print(false_arr)
int_ones = np.ones(4, dtype=int)
print(int_ones)
float_twos = np.full((2, 3), 2.0)
print(float_twos)

### Vectorization

Vectorized code handles arrays as a whole instead of looping through their elements and handling them individually. Suppose we have an array and we wish to create a new array with the values doubled.

In [ ]:
def double_not_vectorized(arr):
    return [2 * elem for elem in arr]


def double_vectorized(arr):
    return 2 * arr


small_list = [1, 4, 9]
print(double_not_vectorized(small_list))
small_array = np.array(small_list)
print(double_vectorized(small_array))

Here is another example where we increment the elements by one.

In [ ]:
print(f"Initial list: {small_list}")

# Normal Python with list comprehension
incremented_list = [elem + 1 for elem in small_list]

print(f"Incremented list: {incremented_list}")
print()

# numpy
print(f"Initial array: {small_array}")
print(f"Incremented array: {small_array+1}")

Many `numpy` functions can also be applied to arrays element-wise.

In [ ]:
angles = np.arange(0, 361, 45)
print(f"Angles are {angles} degrees")
angles = np.deg2rad(angles)
print(f"Angles are {angles} radians")
print(f"Cosines are {np.cos(angles)}")

Some functions can be applied to the array as a whole or along some specific axis.

In [ ]:
print(f"Array:\n{diffs}")
print(f"Total sum: {np.sum(diffs)}")
for i in range(2):
    print(f"Sums along axis {i}: {np.sum(diffs, axis=i)}")
print(f"Maximum value: {np.max(diffs)}")
for i in range(2):
    print(f"Maximum values along axis {i}: {np.max(diffs, axis=i)}")

### Slicing

Slicing `numpy` arrays uses syntax similar to slicing basic Python sequences, but arrays can be sliced independently along different axes.

In [ ]:
print(diffs, diffs[1:], diffs[::2], diffs[1:, :-1], diffs[:, 2], sep="\n\n")

Note that `numpy` arrays with multiple dimensions are accessed with tuples that specify (or not) the indices in the dimension, i.e `diffs[i,j]` for i-th row and j-th column.
Something similar can be constructed with Python lists, where the `list` of rows each contain a column `list`.
These are then accessed by first indexing the row and then the column like `diffs[i][j]`. 

### Masking

Sometimes we wish to perform operations only on a subset of array elements that satisfy some condition. This can be achieved with masking. Suppose we have an array of integers and we wish to double its odd values but leave the even values unchanged.

In [ ]:
print(f"Original array:\n{diffs}")
mod_diffs = diffs.copy()
mod_diffs[diffs % 2 == 1] *= 2
print(f"Modified array:\n{mod_diffs}")

In the above code the expression `diffs%2 == 1` creates a mask of Boolean values. Only the subset of values corresponding to `True` in the mask are doubled.

### Broadcasting

If two arrays have the same shape then it is possible to perform element-wise operations.

In [ ]:
a = np.arange(3)
print(a)
b = np.arange(4, 9, 2)
print(b)
print(a - b)
print(a * b)

Sometimes it is also possible to do this is even if the arrays have different shapes. This is known as [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html) (an illustrated explanation is available [here](https://numpy.org/devdocs/user/theory.broadcasting.html)). We will not repeat the rules of broadcasting here, but we do offer a brief description of how we used it to compute the pairwise distances with the function `pairwise_differences_with_numpy()`.

In [ ]:
# The input array
print(small_array)

# Input is a 1-dimensional array
print(f"{small_array.shape = }")

# The expected output
print(diffs, "\n")

# We can add another axis to the array without changing its elements
column_vector = small_array[:, np.newaxis]

# We now have a column vector
print(column_vector)
print(f"{column_vector.shape = }\n")

# We could also convert the input into a row vector
row_vector = small_array[
    np.newaxis,
]
print(row_vector)
print(f"{row_vector.shape = }\n")

# Broadcasting stretches the column vector to a matrix that has the i-th element in i-th row.
# The row vector gets stretched to a matrix that has the j-th element in the j-th column.
# The difference of these matrices has in its i,j position the difference of elements with
# indices i and j, which is exactly what we want.
print(column_vector - row_vector, "\n")

# We don't have to store the row and column vectors, so we could just write
print(
    small_array[:, np.newaxis]
    - small_array[
        np.newaxis,
    ],
    "\n",
)
# But according to broadcasting rules the second broadcasting can be performed implicitly
print(small_array[:, np.newaxis] - small_array)

The initial array has the shape (3,), so the table of pairwise differences must have the shape (3,3). It could be tempting to convert `small_array` into the correct shape by taking the outer product with an array of ones.

In [ ]:
temp = np.outer(small_array, np.ones(small_array.shape, dtype=int))
print(temp)

Finding the pairwise distances can now be done explicitly if we think of this intermediate 2D array as a matrix and apply the transposing operation.

In [ ]:
print(temp - temp.T)

The problem of this approach is that the temporary matrix ``temp`` needs to be stored in the memory. For a matrix of such a small size this shortcoming is not noticeable, but for larger datasets it could well be. Broadcasting achieves the same outcome without creating and storing temporary matrices and also with less code.

## SciPy

[SciPy](https://www.scipy.org/) contains many useful functions for numerical integration, algebra, Fourier transforms and much more.
Using `scipy` is quite straightforward, especially if you are familiar with `numpy`.
The hardest part is often figuring out which function you should be using, but that problem can usually be solved by an Internet search engine.

## Astropy

[Astropy](https://www.astropy.org/) is a very useful package for using Python in Astronomy.
Here we will limit ourselves to demonstrating only two useful aspects of `astropy`.

### Units

The `Quantity` class in `astropy` can handle [physical quantities](https://docs.astropy.org/en/stable/units/) that have some value in some unit system.
Many physical constants are also built in.

In [ ]:
from astropy import units as u
from astropy.constants import G

angles = np.arange(0, 361, 45) * u.degree
print(f"{angles = }")
print(f"{np.sin(angles) = }")  # We do not have to explicitly convert degrees to radians
print(f"{angles.to(u.rad) = }")  # But we can if we want to
print()

r = 1 * u.au
t = 1 * u.yr
v = 2 * np.pi * r / t
print(f"{v = }")
print(f"{v.to(u.km/u.s) = }")
print()

print(G)

## Tables

`astropy` also implements [Tables](https://docs.astropy.org/en/stable/table/index.html) that allow data to be grouped and handled together.
_An `astropy` `Table` is very similar to a `pandas` `DataFrame`, but it supports multidimensional columns and `astropy` classes such as `SkyCoord`._
 A `QTable` is a `Table` that can have `Quantity` instances as columns.
The following demonstrates some basic functionality.

In [ ]:
from astropy.table import QTable

# Creating a QTable
labels = ["Earth", "Jupiter", "Sun"]
m = [1 * u.M_earth, 1 * u.M_jupiter, 1 * u.M_sun]
r = [1 * u.R_earth, 1 * u.R_jupiter, 1 * u.R_sun]
data = QTable((labels, m, r), names=["name", "mass", "radius"])

print(data)
print()
print(data.info)
print()
print(data.info("stats"))
print()

# It is possible to add new columns
data["density"] = (data["mass"] / (4 * np.pi / 3 * data["radius"] ** 3)).to(
    u.g / u.cm**3
)
print(data)
print()

# We can filter data based on the values of some columns
print(data[data["density"] < 2000 * u.kg / u.m**3])
print()

# We can easilly access data for a specific object
print(data[data["name"] == "Sun"])